In [1]:
import torch
# Check if PyTorch supports the built-in alternative (SDPA)
print(hasattr(torch.nn.functional, 'scaled_dot_product_attention'))

print("Checking for CUDA-enabled GPU...")
print(torch.cuda.is_available()) # Returns True if a CUDA-enabled GPU is detected

print(" Returns the index of the currently selected GPU")
print(torch.cuda.current_device()) # Returns the index of the currently selected GPU)



True
Checking for CUDA-enabled GPU...
True
 Returns the index of the currently selected GPU
0


In [2]:
from ultralytics import YOLO
# Load a YOLOv8 model (you can replace 'yolov8n.pt' with your model path)
model = YOLO('yolov8n.pt')


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/teamspace/studios/this_studio/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Train the model
results = model.train(
    data='data.yaml',  # Path to your config file
    epochs=100,         # 100 runs through the data is a good start
    imgsz=640,         # Resizes images to 640x640 pixels for training
    batch=16,          # How many images to process at once (lower if you run out of memory)
    device=0,           # Use 0 for GPU. Use 'cpu' if you don't have an NVIDIA GPU.
    # workers=4,          # How many CPU cores load data. Set to 0 if you get "broken pipe" errors on Windows.
    # project='pothole_project', # Name of the folder where results are saved
    # name='run_1'        # Name of the sub-folder for this specific experiment
)
# results = model.train(
#     data='data.yaml',  # Path to your config file
#     epochs=1,         # 50 runs through the data is a good start
#     imgsz=640,         # Resizes images to 640x640 pixels for training
#     batch=16,          # How many images to process at once (lower if you run out of memory)
#     device=0           # Use '0' for GPU, or 'cpu' if you don't have an NVIDIA GPU
# )

print("Training complete!")

Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

In [5]:
# Validate the model using the test dataset
from ultralytics import YOLO

# Load your TRAINED model (not the 'yolo12n.pt' starter one!)
# Make sure to point to the 'best.pt' file created during your 2-epoch run.
model = YOLO('runs/detect/train2/weights/best.pt')

# Run Validation
metrics = model.val(
    data='data.yaml', 
    split='test'       # This tells it to use the 'test' folder defined in your yaml
)

print(f"Mean Average Precision (mAP50): {metrics.box.map50}")

Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 85.0±18.7 MB/s, size: 73.9 KB)
val: Scanning /teamspace/studios/this_studio/yolo_project/data/test/labels... 128 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 1.5Kit/s 0.1s
val: New cache created: /teamspace/studios/this_studio/yolo_project/data/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 4.2it/s 1.9s0.4s
                   all        128        203      0.682      0.537      0.565      0.328
Speed: 3.4ms preprocess, 3.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /teamspace/studios/this_studio/yolo_project/runs/detect/val
Mean Average Precision (mAP50): 0.5646431013261981


In [6]:
# Clear GPU Memory Before Loading a New Model
import gc
import torch

# 1. Delete the old model variable if it exists
if 'model' in locals():
    del model

# 2. Force Python to throw away garbage
gc.collect()

# 3. Clear the GPU cache (Crucial step!)
torch.cuda.empty_cache()

print("GPU Memory Cleared! Ready for the new model.")

GPU Memory Cleared! Ready for the new model.


In [7]:
from ultralytics import YOLO

# 1. Load the YOLOv12 Small model (Better balance for T4 GPU)
model = YOLO('yolo12s.pt') 

In [8]:
# Train the model

# 2. Train
results = model.train(
    data='data.yaml',
    epochs=100,         # Keep 100
    imgsz=640,          # Keep 640 for now
    batch=24,           # T4 has 16GB VRAM; you can increase batch size to 24 or 32!
    device=0,           # Use your T4
    patience=15,        # Stop if no improvement for 15 epochs
    project='yolo_project',
    name='train_v12_small' # Name it clearly so you don't confuse it with the v8 run
)
# results = model.train(
#     data='data.yaml',  # Path to your config file
#     epochs=1,         # 50 runs through the data is a good start
#     imgsz=640,         # Resizes images to 640x640 pixels for training
#     batch=16,          # How many images to process at once (lower if you run out of memory)
#     device=0           # Use '0' for GPU, or 'cpu' if you don't have an NVIDIA GPU
# )

print("Training complete!")

Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_v12_small, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=True, pose=12.0, pretrai

In [9]:
# Validate the model using the test dataset
from ultralytics import YOLO

# Load your TRAINED model (not the 'yolo12n.pt' starter one!)
# Make sure to point to the 'best.pt' file created during your 2-epoch run.
model = YOLO('yolo_project/train_v12_small/weights/best.pt')

# Run Validation
metrics = model.val(
    data='data.yaml', 
    split='test'       # This tells it to use the 'test' folder defined in your yaml
)

print(f"Mean Average Precision (mAP50): {metrics.box.map50}")

Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 83.9±32.6 MB/s, size: 73.4 KB)
val: Scanning /teamspace/studios/this_studio/yolo_project/data/test/labels.cache... 128 images, 13 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 283.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.7it/s 2.9s0.3ss
                   all        128        203      0.652      0.522      0.553      0.311
Speed: 3.2ms preprocess, 12.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /teamspace/studios/this_studio/yolo_project/runs/detect/val2
Mean Average Precision (mAP50): 0.5533028709682577


### Attempt to train a fresh model with different parameters ()

In [2]:
# Initialize a FRESH model (resets all weights to standard)
import torch
from ultralytics import YOLO

model = YOLO('yolo12s.pt') 

results = model.train(
    data='data.yaml',
    epochs=300,           # Full duration of 300 epochs
    patience=50,
    batch=32,          # Max for T4
    imgsz=640,
    close_mosaic=10,      # The new hyperparameter
    project='yolo_project',
    name='train_v12s_300_scratch'
)

New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_v12s_300_scratch, nbs=64, nms=False, opset=None,